In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


In [2]:
supply_data = pd.read_csv('/Users/aimaldastagirzada/Downloads/ecogreen_energy_supply.csv')
demand_data = pd.read_csv('/Users/aimaldastagirzada/Downloads/ecogreen_energy_demand.csv')

In [3]:
model = gp.Model("EcoGreen_Energy_Expansion")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [4]:
# Indices for plants and provinces
plants = range(len(supply_data))
provinces = range(len(demand_data))

In [5]:
# Decision Variables
# y_i: Binary decision for building plant i
y = model.addVars(plants, vtype=GRB.BINARY, name="Plant")
# x_ij: Continuous variable for energy from plant i to province j
x = model.addVars(plants, provinces, vtype=GRB.CONTINUOUS, name="Transport")

In [6]:
# Objective Function to minimize total cost
model.setObjective(
    gp.quicksum(
        y[i] * supply_data.loc[i, 'Fixed'] +
        gp.quicksum(x[i, j] * supply_data.loc[i, f'Province {j+1}'] for j in provinces)
        for i in plants),
    GRB.MINIMIZE)

In [7]:
# Constraints
# Capacity constraints for each plant
model.addConstrs(
    (gp.quicksum(x[i, j] for j in provinces) <= supply_data.loc[i, 'Capacity'] * y[i]
     for i in plants), name="Capacity")


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>}

In [8]:
# Demand fulfillment for each province
model.addConstrs(
    (gp.quicksum(x[i, j] for i in plants) >= demand_data.loc[j, 'Demand']
     for j in provinces), name="Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>}

In [9]:
# Logical and additional constraints
model.addConstr(y[9] + y[14] + y[19] <= 1, "Mutual_Exclusivity")
model.addConstr(y[2] <= y[3], "Site3_to_Site4")
model.addConstr(y[2] <= y[4], "Site3_to_Site5")
model.addConstr(y[4] <= y[7] + y[8], "Site5_dependency")
model.addConstr(gp.quicksum(y[i] for i in range(10)) <= 2 * gp.quicksum(y[i] for i in range(10, 20)), "Region_A_vs_B")
total_output = gp.quicksum(x[i, j] for i in plants for j in provinces)
model.addConstr(gp.quicksum(x[i, j] for i in range(5) for j in provinces) >= 0.3 * total_output, "Minimum_Output_Sites_1_5")


<gurobi.Constr *Awaiting Model Update*>

In [10]:
# Single plant output cap
for i in plants:
    for j in provinces:
        model.addConstr(x[i, j] <= 0.5 * demand_data.loc[j, 'Demand'], f"Cap_{i}_{j}")


In [11]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 236 rows, 220 columns and 850 nonzeros
Model fingerprint: 0x0cc4d4af
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 2.938222e+08
Presolve removed 200 rows and 0 columns
Presolve time: 0.01s
Presolved: 36 rows, 220 columns, 650 nonzeros
Variable types: 200 continuous, 20 integer (20 binary)

Root relaxation: objective 1.685904e+08, 67 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6859e+08    0    4 2.9382e+08 1.6859e+08  42.6

In [13]:
# Display the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    # Print the optimal total cost
    print(f"Total Optimal Cost: ${model.ObjVal:,.2f}")

    # Print the decision to build each plant
    for i in plants:
        print(f"Plant {i+1} built: {'Yes' if y[i].X > 0.5 else 'No'}")

    # Print energy sent from each plant to each province, where applicable
    for i in plants:
        for j in provinces:
            if x[i, j].X > 0:
                print(f"Energy from Plant {i+1} to Province {j+1}: {x[i, j].X} GWh")
else:
    print("Optimal solution not found. Status code:", model.status)


Optimal solution found.
Total Optimal Cost: $193,005,688.79
Plant 1 built: Yes
Plant 2 built: Yes
Plant 3 built: Yes
Plant 4 built: Yes
Plant 5 built: Yes
Plant 6 built: No
Plant 7 built: Yes
Plant 8 built: Yes
Plant 9 built: No
Plant 10 built: No
Plant 11 built: Yes
Plant 12 built: No
Plant 13 built: No
Plant 14 built: No
Plant 15 built: Yes
Plant 16 built: Yes
Plant 17 built: Yes
Plant 18 built: No
Plant 19 built: No
Plant 20 built: No
Energy from Plant 1 to Province 4: 81571.5 GWh
Energy from Plant 1 to Province 10: 43688.5 GWh
Energy from Plant 2 to Province 1: 49395.5 GWh
Energy from Plant 2 to Province 8: 85696.5 GWh
Energy from Plant 3 to Province 7: 42728.5 GWh
Energy from Plant 3 to Province 9: 26438.5 GWh
Energy from Plant 4 to Province 6: 57834.0 GWh
Energy from Plant 5 to Province 3: 62753.0 GWh
Energy from Plant 5 to Province 6: 11290.5 GWh
Energy from Plant 5 to Province 7: 5469.5 GWh
Energy from Plant 7 to Province 3: 62753.0 GWh
Energy from Plant 7 to Province 9: 63863.

In [14]:
# After solving the model, calculate the number of provinces each plant supplies
for i in plants:
    num_provinces_supplied = sum(1 for j in provinces if x[i, j].X > 0)
    print(f"Plant {i+1} supplies energy to {num_provinces_supplied} distinct provinces.")


Plant 1 supplies energy to 2 distinct provinces.
Plant 2 supplies energy to 2 distinct provinces.
Plant 3 supplies energy to 2 distinct provinces.
Plant 4 supplies energy to 1 distinct provinces.
Plant 5 supplies energy to 3 distinct provinces.
Plant 6 supplies energy to 0 distinct provinces.
Plant 7 supplies energy to 3 distinct provinces.
Plant 8 supplies energy to 2 distinct provinces.
Plant 9 supplies energy to 0 distinct provinces.
Plant 10 supplies energy to 0 distinct provinces.
Plant 11 supplies energy to 3 distinct provinces.
Plant 12 supplies energy to 0 distinct provinces.
Plant 13 supplies energy to 0 distinct provinces.
Plant 14 supplies energy to 0 distinct provinces.
Plant 15 supplies energy to 3 distinct provinces.
Plant 16 supplies energy to 3 distinct provinces.
Plant 17 supplies energy to 3 distinct provinces.
Plant 18 supplies energy to 0 distinct provinces.
Plant 19 supplies energy to 0 distinct provinces.
Plant 20 supplies energy to 0 distinct provinces.


In [15]:
# Total number of decision variables
total_decision_variables = len(y) + len(x)
print(f"Total decision variables required: {total_decision_variables}")

# Using Gurobi to report the number of variables
print(f"Gurobi reports {model.NumVars} variables.")

Total decision variables required: 220
Gurobi reports 220 variables.


In [16]:
# Displaying capacity constraints
for constr in model.getConstrs():
    if "Capacity" in constr.ConstrName:
        print(f"{constr.ConstrName}: {constr}")

# Counting capacity constraints
print(f"Total capacity constraints: {sum(1 for constr in model.getConstrs() if 'Capacity' in constr.ConstrName)}")

Capacity[0]: <gurobi.Constr Capacity[0]>
Capacity[1]: <gurobi.Constr Capacity[1]>
Capacity[2]: <gurobi.Constr Capacity[2]>
Capacity[3]: <gurobi.Constr Capacity[3]>
Capacity[4]: <gurobi.Constr Capacity[4]>
Capacity[5]: <gurobi.Constr Capacity[5]>
Capacity[6]: <gurobi.Constr Capacity[6]>
Capacity[7]: <gurobi.Constr Capacity[7]>
Capacity[8]: <gurobi.Constr Capacity[8]>
Capacity[9]: <gurobi.Constr Capacity[9]>
Capacity[10]: <gurobi.Constr Capacity[10]>
Capacity[11]: <gurobi.Constr Capacity[11]>
Capacity[12]: <gurobi.Constr Capacity[12]>
Capacity[13]: <gurobi.Constr Capacity[13]>
Capacity[14]: <gurobi.Constr Capacity[14]>
Capacity[15]: <gurobi.Constr Capacity[15]>
Capacity[16]: <gurobi.Constr Capacity[16]>
Capacity[17]: <gurobi.Constr Capacity[17]>
Capacity[18]: <gurobi.Constr Capacity[18]>
Capacity[19]: <gurobi.Constr Capacity[19]>
Total capacity constraints: 20


In [20]:
##(d) 
print("Constraint for mutual exclusivity between sites 10, 15, and 20:")
print(model.getConstrByName("Mutual_Exclusivity"))

Constraint for mutual exclusivity between sites 10, 15, and 20:
<gurobi.Constr Mutual_Exclusivity>


In [21]:
##e
print("Dependency constraint for site 5 on sites 8 or 9:")
print(model.getConstrByName("Site5_dependency"))

Dependency constraint for site 5 on sites 8 or 9:
<gurobi.Constr Site5_dependency>


In [22]:
##F
if model.status == GRB.OPTIMAL:
    print(f"Optimal Cost: ${model.ObjVal:,.2f}")
else:
    print("Optimal solution was not found.")


Optimal Cost: $193,005,688.79


In [23]:
##g
num_plants_established = sum(1 for i in plants if y[i].X > 0.5)
print(f"Number of power plants established: {num_plants_established}")

Number of power plants established: 11


In [24]:
##H
# Calculate the number of plants supplying each province
plants_per_province = {j: sum(1 for i in plants if x[i, j].X > 0) for j in provinces}

# Finding the highest and lowest
max_plants = max(plants_per_province.values())
min_plants = min(plants_per_province.values())
print(f"Highest number of plants for a province: {max_plants}")
print(f"Lowest number of plants for a province: {min_plants}")

Highest number of plants for a province: 4
Lowest number of plants for a province: 2


In [25]:
##J
# Set MIP gap to 1%
model.Params.MIPGap = 0.01

# Optimize the model
model.optimize()

# Reporting the number of feasible solutions found within 1% of the optimal cost
print("Number of feasible solutions within 1% of the optimal solution:", model.SolCount)

Set parameter MIPGap to value 0.01
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 236 rows, 220 columns and 850 nonzeros
Model fingerprint: 0x0cc4d4af
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolved: 36 rows, 220 columns, 650 nonzeros

Continuing optimization...


Explored 1 nodes (95 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 6: 1.93006e+08 1.93021e+08 1.97538e+08 ... 2.93822e+08

Optimal solution found (tolerance 1.00e-02)
Best objective 1.930056887899e+08, best bound 1.929875920156e+08, gap 0.0094%
Number of feasible solutions within 1% of the optimal solution: 6


In [26]:
##k
# Remove constraints limiting plant output to 50% of province needs
for i in plants:
    for j in provinces:
        model.remove(model.getConstrByName(f"Cap_{i}_{j}"))

model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Cost without 50% cap: ${model.ObjVal:,.2f}")
    print(f"Number of plants established without 50% cap: {sum(1 for i in plants if y[i].X > 0.5)}")
else:
    print("Optimal solution was not found after removing the 50% cap constraints.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 36 rows, 220 columns and 650 nonzeros
Model fingerprint: 0xc01f898d
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]

MIP start from previous solve produced solution with objective 1.92981e+08 (0.04s)
Loaded MIP start from previous solve with objective 1.92981e+08

Presolve time: 0.00s
Presolved: 36 rows, 220 columns, 650 nonzeros
Variable types: 200 continuous, 20 integer (20 binary)

Root relaxation: objective 1.685643e+08, 61 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  